In [1]:
import pymongo
import string
from tqdm.notebook import tqdm
from collections import Counter 

client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['vatican']
dataset = db['perturbedDataset']
wordsDataset = db['wordsDataset']

docs = list(dataset.find())

In [2]:
#wordsDataset.drop()
wordsDataset.count()

<ipython-input-2-ab340a24988d>:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  wordsDataset.count()


0

In [3]:
chars = "solenità'd\mczkwfhjuxgbvrpyq8192!#"
CHAR_INDEX = dict((c, i) for i, c in enumerate(chars))

def preprocess(s):
    out = ""
    for c in s.lower():
        if c in CHAR_INDEX.keys():
            out += c
        else:
            out += '#'
    return "".join(out)

In [4]:
word = dict()

for i, d in tqdm(enumerate(docs)):
    for j, s in enumerate(d['sentences']):
        for k, w in enumerate(s[0]):
            word.setdefault(s[1][k], []).append(preprocess(w))            
            
for wk in tqdm(word.keys()):
    item = dict(Counter(word.get(wk)))
    wordsDataset.insert_one({'word': preprocess(wk), 'occurrences': item})
    
wordsDataset.find_one()

{'_id': ObjectId('60eabb6a6efb0ca2764b2913'),
 'word': 'solennità',
 'occurrences': {'solennità': 41, "solennita'̀": 2}}

In [5]:
for i in wordsDataset.find({'word': 'ha'}):
    print(i)

{'_id': ObjectId('60eabb6b6efb0ca2764b2a89'), 'word': 'ha', 'occurrences': {'ha': 199, 'hta': 1, 'a': 1, 'hp': 1, 'hb': 1, 'hk': 1}}


In [7]:
a = list(wordsDataset.find())
for i in a:
    if(i['word'] == 'ha'):
        print(i)

{'_id': ObjectId('60eabb6b6efb0ca2764b2a89'), 'word': 'ha', 'occurrences': {'ha': 199, 'hta': 1, 'a': 1, 'hp': 1, 'hb': 1, 'hk': 1}}
